In [1]:
#ConfettiBot, por Luis García Nieto

#Captura pantalla
import pyscreenshot as ImageGrab

#Lectura de texto en imagenes por tesseract ocr
from PIL import Image
import pytesseract
#API de google para buscar por medio de terminal
from googleapiclient.discovery import build

#(Opcional) Para seleccionar la respuesta de manera automática
import win32api, win32con

In [3]:
#Credenciales del buscador personalizado de Google
my_key= #Inserta tu Google API key
my_id= #Inserta la ID de tu Custom Search Engine

#Camino a donde se encuentra instalado Google Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract"

In [4]:
#Busca search_term en Google
def google_search(search_term, api_key, cse_id, mode, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res[mode]

#Da click en coordenadas x,y de la pantalla
def click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)

#Modificar los parametros de click a la posición de las posibles respuestas
def select(option):
    if option==0:
        click(740,580)
        print("a")
    if option==1:
        print("b")
        #Good position
        click(740,650)
    if option==2:
        click(740,710)
        print("c")

In [ ]:
import time
start=time.time()

#Captura la región donde se encuentran la pregunta y las respuestas
qim=ImageGrab.grab(bbox=(700,470,1030,735))
qim.save("t.png")
#Lee el texto en la captura
ocr=pytesseract.image_to_string(Image.open("t.png"),lang='spa')

print(time.time()-start)

#Horrible manipulación de los datos
#Muy probablemente hay un error aquí  ¯\_(ツ)_/¯
#Nada optimizado
data=[x.strip() for x in ocr.splitlines()]
question=" ".join(data[0:len(data)-6])
print(question)
print("\n")

answers=data[-5:]
answers=[x for x in answers if len(x)>0]
ans=[x.split(" ") for x in answers]
ans=[[s for s in ans[i] if len(s)>2]for i in range(3)]
ans=[" ".join(y) for y in ans]


#Método alternativo de respuesta, buscar la respuesta en los resultados de Google de la pregunta
#Algo lento y no era mucho más acertado
"""
results = google_search(question, my_key, my_id,"items", num=10)
dic={}

for answer in ans:
    count=0
    for result in results:
        if answer.lower() in result["snippet"].lower():
            count=count+1
            print(answer)
        if answer.lower() in result["title"].lower():
            count=count+1
            print(answer)
    dic[answer]=count

print(dic)
print(time.time()-start)
"""
#Método de respuesta, 
nums=[]
for answer in answers:
    temp = google_search(question+answer, my_key, my_id,"searchInformation", num=10)
    print(temp)
    nums.append(int(temp['totalResults']))
    #Quedando poco tiempo selecciona A por default
    if time.time()-start>9:
        select(0)

select(nums.index(max(nums)))
print(time.time()-start)